# Analysis of the approximation

Copyright 2025 Stephan Kulla ("Kulla")

Licensed under the Apache License, Version 2.0 (the "Apache License") and Creative Commons Attribution 4.0 International (the "CC-BY License"). You may choose either of these licenses to govern your use of this project.

You may obtain a copy of the Apache License at:
    http://www.apache.org/licenses/LICENSE-2.0

You may obtain a copy of the CC-BY License at:
    https://creativecommons.org/licenses/by/4.0/

Unless required by applicable law or agreed to in writing, software and content distributed under the Apache License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the Apache License for the specific language governing permissions and limitations under the License.

Under the CC-BY License, you are free to share and adapt the material provided you give appropriate credit, provide a link to the license, and indicate if changes were made. See the CC-BY License for full details.

Third-Party Components and Licenses:
This product also includes third-party components which are distributed under their respective licenses. Please refer to the NOTICE file for details.

## Utility functions and imports

In [1]:
%load_ext jupyter_ai_magics

In [2]:
%load_ext autoreload

In [4]:
%autoreload 2

In [5]:
import string
import sympy as sp

from IPython.display import display, Markdown

class LatexFormatter(string.Formatter):
    def format_field(self, value, format_spec):
        if format_spec == 'tex':
            return sp.printing.latex(value)
        if format_spec == '$' or format_spec == 'inline':
            return "$" + sp.printing.latex(value) + "$"
        elif format_spec == '$$' or format_spec == 'block':
            return "$$" + sp.printing.latex(value) + "$$"
        else:
            return super().format_field(value, format_spec)

def disp_md(template, **kwargs):
    display(Markdown(LatexFormatter().format(template, **kwargs)))

## Analysis

In [14]:
from sympy.abc import x,n
from src.approximation import matrix_of
from src.c_space import vectors, integral_forms

def approximation(linear_forms, vectors, func):
    matrix = matrix_of(linear_forms, vectors)
    coefficients = matrix.solve(sp.Matrix([lf(func) for lf in linear_forms]))

    return sum(coeff * basis for coeff, basis in zip(coefficients, vectors))

a = approximation(integral_forms(sp.legendre(n, x)*sp.sqrt((2*n+1)/2), 5), vectors(sp.legendre(n, x)*sp.sqrt((2*n+1)/2), 5), sp.sin(x))
b = approximation(integral_forms((1-x)**n/sp.factorial(n), 5), vectors(x**n, 5), sp.sin(x))

display(a)
display(b)

a.equals(b)

sqrt(6)*x*(-26015238*sqrt(6) - 25078834*sqrt(6)*cos(1) + 47019316*sqrt(6)*sin(1))/2 + sqrt(10)*(3*x**2/2 - 1/2)*(-29745552*sqrt(10)*sin(1) + 15899312*sqrt(10)*cos(1) + 16439584*sqrt(10))/2 + sqrt(14)*(5*x**3/2 - 3*x/2)*(-7869728*sqrt(14) - 7653584*sqrt(14)*cos(1) + 14266656*sqrt(14)*sin(1))/2 + 3*sqrt(2)*(35*x**4/8 - 15*x**2/4 + 3/8)*(-14247840*sqrt(2)*sin(1) + 7690080*sqrt(2)*cos(1) + 7834176*sqrt(2))/2 + sqrt(22)*(-470432*sqrt(22) - 470432*sqrt(22)*cos(1) + 861120*sqrt(22)*sin(1))*(63*x**5/8 - 35*x**3/4 + 15*x/8)/2 + sqrt(2)*(-58637670*sqrt(2)*sin(1) + 31244786*sqrt(2)*cos(1) + 32460268*sqrt(2))/2

x**5*(-40751172 - 40751172*cos(1) + 74594520*sin(1)) + x**4*(-187002900*sin(1) + 100932300*cos(1) + 102823560) + x**3*(-92441160 - 88658640*cos(1) + 166783680*sin(1)) + x**2*(-62803440*sin(1) + 32731440*cos(1) + 35162400) + x*(-5116230 - 4576530*cos(1) + 9018660*sin(1)) - 302610*sin(1) + 147846*cos(1) + 174756

True